In [1]:
import pandas as pd

class DynamicJSON:
    def __init__(self, data):
        self.data = data
        self.children = {}
        self._process_data(data)

    def _process_data(self, data):
        """
        Dynamically process JSON data and set attributes.
        Nested lists and dictionaries are converted to DynamicJSON objects or DataFrames.
        """
        if isinstance(data, dict):
            for key, value in data.items():
                if isinstance(value, dict):
                    # Recursively create a DynamicJSON object for nested dictionaries
                    self.children[key] = DynamicJSON(value)
                elif isinstance(value, list):
                    # Recursively process lists; convert to DataFrames if possible
                    self.children[key] = pd.DataFrame(
                        [DynamicJSON(item).to_dict() if isinstance(item, dict) else {'value': item} for item in value]
                    )
                else:
                    # Set other types as attributes directly
                    setattr(self, key, value)

    def to_dict(self):
        """
        Convert the object and its children back into a dictionary.
        """
        result = {}
        for key, value in self.children.items():
            if isinstance(value, DynamicJSON):
                result[key] = value.to_dict()
            elif isinstance(value, pd.DataFrame):
                result[key] = value.to_dict(orient="records")
        return {**result, **self.__dict__}

    def __repr__(self):
        return f"<DynamicJSON: {list(self.children.keys()) + list(self.__dict__.keys())}>"

# Example JSON structure
json_data = {
    'deckid': 'a',
    'createdat': '2024-10-01',
    'deckCards': [
        {
            'cardname': 'TEST',
            'cardvalue': 3,
            'priceperprinting': [{'printname': 'printA', 'value': 3}, {'printname': 'printB', 'value': 4}],
        },
        {
            'cardname': 'TEST2',
            'cardvalue': 3,
            'priceperprinting': [{'printname': 'printA', 'value': 3}, {'printname': 'printB', 'value': 4}],
        },
    ],
}

# Create a DynamicJSON object
dynamic_object = DynamicJSON(json_data)

# Accessing attributes
print("Deck ID:", dynamic_object.deckid)
print("Created At:", dynamic_object.createdat)

# Access nested DataFrames
deck_cards_df = dynamic_object.children['deckCards']
print("\nDeck Cards DataFrame:")
print(deck_cards_df)

# Access deeper nested DataFrames (e.g., priceperprinting for the first card)
first_card_prices_df = deck_cards_df['priceperprinting']
print("\nFirst Card PricePerPrinting DataFrame:")
print(first_card_prices_df)

# Convert back to JSON
print("\nReconstructed JSON:")
print(dynamic_object.to_dict())


Deck ID: a
Created At: 2024-10-01

Deck Cards DataFrame:
                                    priceperprinting  \
0  [{'data': {'printname': 'printA', 'value': 3},...   
1  [{'data': {'printname': 'printA', 'value': 3},...   

                                                data  \
0  {'cardname': 'TEST', 'cardvalue': 3, 'priceper...   
1  {'cardname': 'TEST2', 'cardvalue': 3, 'pricepe...   

                                     children cardname  cardvalue  
0  {'priceperprinting': ['data', 'children']}     TEST          3  
1  {'priceperprinting': ['data', 'children']}    TEST2          3  

First Card PricePerPrinting DataFrame:
[{'data': {'printname': 'printA', 'value': 3}, 'children': {}, 'printname': 'printA', 'value': 3}, {'data': {'printname': 'printB', 'value': 4}, 'children': {}, 'printname': 'printB', 'value': 4}]

Reconstructed JSON:
{'deckCards': [{'priceperprinting': [{'data': {'printname': 'printA', 'value': 3}, 'children': {}, 'printname': 'printA', 'value': 3}, {'data':

In [8]:
dynamic_object.children['deckCards']['priceperprinting'].iloc[0]

[{'data': {'printname': 'printA', 'value': 3},
  'children': {},
  'printname': 'printA',
  'value': 3},
 {'data': {'printname': 'printB', 'value': 4},
  'children': {},
  'printname': 'printB',
  'value': 4}]